# N-Gram Playground

N-Gram language models model the structure of a language by modeling the probabilities of words depending on a history of the previous n - 1 words. This simple approach, with a statistical model that is extremely easy to estimate from some givent text, nevertheless manages to capture a good deal of what makes language language.

N-Gram language models can be used to estimate the probability of a given sentence ("how likely am I to see this sentence in this language"). They can also be used generatively, by starting with a seed phrase and then picking additional words according to the probability the n-gram LM would assign them given the current context.

This file contains some code to estimate an n-gram LM from a given text or some hard-coded sentences, which can then be used to estimate the probability of a sentence. It also contains code for generating text, which can also be used to verify the probability of a sentence actually appearing.

# Settings
order gives the maximum order of the n-grams (i.e. the n in n-gram). readText sets whether to use hard-coded text for training, or to read text in from a file (The suggested file can be found at http://textfiles.com/100/hack11a.txt)

In [1]:
order = 2
readText = False

# Preparing training data

In [2]:
import re
import numpy as np
import pprint

if readText == False:
    # Hardcoded sentences
    trainData = ["john read her book", "i read a different book", "john read a book by mulan"]
else:
    # From http://textfiles.com/100/hack11a.txt
    trainDataFile = "hack11a.txt" # Hacker Crackdown, by Bruce Sterling, January, 1994 [textfiles.com / project gutenberg]
    
    # Read and clean a bit (remove special chars, newlines, ...)
    with open(trainDataFile, 'r') as myfile:
        trainDataString = myfile.read().replace('\n', ' ')
    trainDataString = trainDataString.lower()
    trainDataString = trainDataString.replace('"', "")
    trainDataString = trainDataString.replace(',', "")
    trainDataString = trainDataString.replace(';', "")
    trainDataString = trainDataString.replace(':', "")
    trainDataString = trainDataString.replace(':', "")
    trainDataString = trainDataString.replace('(', "")
    trainDataString = trainDataString.replace(')', "")
    trainDataString = trainDataString.replace('-', " ")
    trainDataString = re.sub("\s\s+" , " ", trainDataString)
    trainData = trainDataString.split(".")

# Calculate n-gram probabilities

This is done iteratively: First, unigram (1-gram) probabilities are estimated, then, using those, bigram (2-gram) probabilities, then trigram probabilities etc., up to the desired order.

In [3]:
tokenizedData = []
for sentence in trainData:
    tokenizedData.append(["<s>"] + sentence.strip().split() + ["</s>"])
    
ngrams = []
ngramCounts = []
for n in range(order):
    thisnGramCounts = {}
    totalCount = 0
    for sentence in tokenizedData:
        for ngramStart in range(len(sentence) - n):
            ngram = sentence[ngramStart:ngramStart + n + 1]
            ngramStr = " ".join(ngram)
            if not ngramStr in thisnGramCounts:
                thisnGramCounts[ngramStr] = 0
            thisnGramCounts[ngramStr] += 1
            totalCount += 1
    ngramCounts.append(thisnGramCounts)
    
    thisNgrams = {}
    thisNgramsByPrefix = {}
    ngramDivisor = totalCount
    for ngramStr in thisnGramCounts.keys():
        ngramPrefix = " ".join(ngramStr.split()[:-1])
        if n != 0:
            ngramDivisor = ngramCounts[n - 1][ngramPrefix]
        ngramProb = ngramCounts[n][ngramStr] / ngramDivisor
        thisNgrams[ngramStr] = ngramProb
    ngrams.append(thisNgrams)
    
if readText == False:
    pprint.pprint(ngrams)

[{'</s>': 0.14285714285714285,
  '<s>': 0.14285714285714285,
  'a': 0.09523809523809523,
  'book': 0.14285714285714285,
  'by': 0.047619047619047616,
  'different': 0.047619047619047616,
  'her': 0.047619047619047616,
  'i': 0.047619047619047616,
  'john': 0.09523809523809523,
  'mulan': 0.047619047619047616,
  'read': 0.14285714285714285},
 {'<s> i': 0.3333333333333333,
  '<s> john': 0.6666666666666666,
  'a book': 0.5,
  'a different': 0.5,
  'book </s>': 0.6666666666666666,
  'book by': 0.3333333333333333,
  'by mulan': 1.0,
  'different book': 1.0,
  'her book': 1.0,
  'i read': 1.0,
  'john read': 1.0,
  'mulan </s>': 1.0,
  'read a': 0.6666666666666666,
  'read her': 0.3333333333333333}]


# Settings for evaluation

The testSentence is the sentence for which the probability is to be calculated.

In [4]:
testSentence = "john read a book"
#testSentence = "hackers are mostly from computer companies"

# Evaluation of probabilities

Note that if the test sentence contains n-grams not seen in training, this will break. There are techniques to mitigate this, but they are not implemented here.

In [5]:
sentenceProb = 1.0
testSentenceTokenized = ["<s>"] + testSentence.split() + ["</s>"]
for ngramStart in range(len(testSentenceTokenized) - order + 1):
    ngram = testSentenceTokenized[ngramStart:ngramStart + order]
    ngramStr = " ".join(ngram)
    print("Probability of n-gram \"" + ngramStr + "\" is " + str(ngrams[order - 1][ngramStr]))
    sentenceProb *= ngrams[order - 1][ngramStr]
print("Probability of sentence \"" + testSentence + "\" is " + str(sentenceProb))

Probability of n-gram "<s> john" is 0.6666666666666666
Probability of n-gram "john read" is 1.0
Probability of n-gram "read a" is 0.6666666666666666
Probability of n-gram "a book" is 0.5
Probability of n-gram "book </s>" is 0.6666666666666666
Probability of sentence "john read a book" is 0.14814814814814814


# Index by prefix

Indexes the n-grams by prefix so the generation step can quickly pick the right n-grams for a given context

In [6]:
ngramsByPrefix = {}
for ngramStr in ngrams[order - 1].keys():
    ngramPrefix = " ".join(ngramStr.split()[:-1])
    if ngramPrefix not in ngramsByPrefix:
        ngramsByPrefix[ngramPrefix] = {}
    ngramsByPrefix[ngramPrefix][ngramStr] = ngrams[order - 1][ngramStr]

# Generation settings

howMany controls how many sentences are generated. The seed is the initial context from which to  start. Note that the prefix has to be at least n - 1 words long for this to work. Try cranking the number of generated sentences up to see the observed probability of encountering the test sentence approach the value calculated for it above.

In [7]:
howMany = 10
seed = "<s>"

# Generate sentences

In [8]:
testSentenceSeenCount = 0
for i in range(howMany):
    result = seed
    while(result.split()[-1] != "</s>"):
        ngramPrefix = " ".join(result.split()[-order+1:])
        potentialNextNgrams = ngramsByPrefix[ngramPrefix]
        potentialNextNgramsStrs = list(potentialNextNgrams.keys())
        potentialNextNgramsProbs = np.array(list(potentialNextNgrams.values()))
        potentialNextNgramsProbs = potentialNextNgramsProbs / np.sum(potentialNextNgramsProbs)
        ngramIndex = np.random.choice(potentialNextNgramsProbs.shape[0], 1, p = potentialNextNgramsProbs)[0]
        nextNgramSuffix = " ".join(potentialNextNgramsStrs[ngramIndex].split()[-1:])
        result += " " + nextNgramSuffix
    
    result = " ".join(result.split()[1:-1])
    if howMany < 100:
        print(result)
        
    if result == testSentence:
        testSentenceSeenCount += 1
        
print("Observed likelihood of test sentence: " + str(testSentenceSeenCount / howMany))

john read her book by mulan
john read a book by mulan
john read a book
john read her book
i read a book
john read a book
i read a book by mulan
i read a book by mulan
john read a different book
i read her book by mulan
Observed likelihood of test sentence: 0.2
